In [1]:
!pip install tensorflow


In [10]:
from google.colab import files
uploaded = files.upload()


Saving charity_data.csv to charity_data.csv


In [13]:
import pandas as pd

# Load data
for filename in uploaded.keys():
    print(f'User uploaded file "{filename}" with length {len(uploaded[filename])} bytes')

# Load the dataset using the uploaded filename
df = pd.read_csv(filename)

# Display the first few rows of the dataframe
df.head()

# Display the column names
print(df.columns)



User uploaded file "charity_data.csv" with length 3605168 bytes
Index(['EIN', 'NAME', 'APPLICATION_TYPE', 'AFFILIATION', 'CLASSIFICATION',
       'USE_CASE', 'ORGANIZATION', 'STATUS', 'INCOME_AMT',
       'SPECIAL_CONSIDERATIONS', 'ASK_AMT', 'IS_SUCCESSFUL'],
      dtype='object')


In [16]:
# Drop the non-beneficial ID columns (replace with actual column names)
df = df.drop(columns=['EIN', 'NAME'])  # Example column names

# Display the first few rows after dropping the columns to verify
df.head()


,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [17]:
# Check unique values for each column
print(df.nunique())

# Combine rare categorical variables (update column name as needed)
application_counts = df['APPLICATION_TYPE'].value_counts()  # Replace 'APPLICATION_TYPE' with the actual column name if different
rare_applications = application_counts[application_counts < 100].index
df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].apply(lambda x: 'Other' if x in rare_applications else x)

# Encode categorical variables
df = pd.get_dummies(df)

# Split into features and target arrays
X = df.drop(columns=['IS_SUCCESSFUL'])  # Ensure 'IS_SUCCESSFUL' is the target column
y = df['IS_SUCCESSFUL']

# Split the data into training and testing datasets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Define the model
model = Sequential()

# Add first hidden layer
model.add(Dense(80, input_dim=X_train_scaled.shape[1], activation='relu'))

# Add second hidden layer
model.add(Dense(30, activation='relu'))

# Add output layer
model.add(Dense(1, activation='sigmoid'))

# Print model summary
model.summary()

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                8800      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 11261 (43.99 KB)
Trainable params: 11261 (43.99 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
# Train the model
model.fit(X_train_scaled, y_train, epochs=100, callbacks=[tf.keras.callbacks.ModelCheckpoint('AlphabetSoupCharity.h5', save_weights_only=True, save_freq=5)])


Epoch 1/100
858/858 [==============================] - 5s 4ms/step - loss: 0.5743 - accuracy: 0.7200
Epoch 2/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5530 - accuracy: 0.7302
Epoch 3/100
858/858 [==============================] - 3s 4ms/step - loss: 0.5504 - accuracy: 0.7309
Epoch 4/100
858/858 [==============================] - 4s 5ms/step - loss: 0.5476 - accuracy: 0.7311
Epoch 5/100
858/858 [==============================] - 3s 4ms/step - loss: 0.5466 - accuracy: 0.7328
Epoch 6/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5454 - accuracy: 0.7326
Epoch 7/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5438 - accuracy: 0.7348
Epoch 8/100
858/858 [==============================] - 4s 4ms/step - loss: 0.5438 - accuracy: 0.7327
Epoch 9/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5422 - accuracy: 0.7338
Epoch 10/100
858/858 [==============================] - 3s 3ms/step - loss: 0.5415 - accura

In [20]:
# Evaluate the model
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")


215/215 [==============================] - 2s 5ms/step - loss: 0.5653 - accuracy: 0.7316
Loss: 0.5652939677238464, Accuracy: 0.731632649898529


In [21]:
# Save the model
model.save('AlphabetSoupCharity.h5')


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
